### Using pyspark to create a logistic regression model to predict whether a passenger will survive or not. 

By: Matt Purvis

In [0]:
# Import SparkSession
from pyspark.sql import SparkSession

In [0]:
# Create the Spark Session
spark = SparkSession.builder.appName('Titanic').getOrCreate()

In [0]:
# Load in the data using Spark SQL
df = spark.sql('select * from titanic_csv')

In [0]:
# Print the schema to get column names and datatypes
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
# Preview the dataset
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| E46| S|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| null| S|
 9| 1| 3|Johnson, Mrs. Osc...|female|27.0| 0| 2| 347742|11.1333| null| S|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| null| C|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| G6| S|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| C103| S|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| null| S|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| null| S|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| null| S|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| null| S|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13.0| null| S|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| null| S|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 20 rows

In [0]:
# Get list of column names
df.columns

Out[4]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
# Filter the df down to columns we are interested in
my_cols = df.select([
 'Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [0]:
# Drop missing values - different strategies for handling missing data. We will drop for now. 
my_final_data = my_cols.na.drop()

In [0]:
# Import classes we will need to encode categorical variables and transform features
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [0]:
# Create objects to use later in the pipeline - will encode categorical variable sex
gender_indexer = StringIndexer(inputCol='Sex', outputCol = 'SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol = 'SexVec')

In [0]:
# Create objects to use later in the pipeline - will encode categorical variable embark
embark_indexer = StringIndexer(inputCol='Embarked', outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol = 'EmbarkVec')

In [0]:
# Create assembler object that will transform the features into a vector - to be used in pipeline later
assembler = VectorAssembler(inputCols = ['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch', 'Fare'],
                           outputCol='features')

In [0]:
# import LogisticRegression class
from pyspark.ml.classification import LogisticRegression

In [0]:
# Import pipeline class
from pyspark.ml import Pipeline

In [0]:
# Create logistic regression model
log_reg_titanic = LogisticRegression(featuresCol = 'features', labelCol = 'Survived')

In [0]:
# Create pipeline that will encode categorical variables sex and embark as well as create the feature vector and the logistic regression model
pipeline = Pipeline(stages = [gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [0]:
# Split data into train and test - could take this further and create a validation set
train_data, test_data = my_final_data.randomSplit([.7,.3])

In [0]:
# Fit the pipeline to the training data
fit_model = pipeline.fit(train_data)

In [0]:
# Use pipeline to make same transformations on the test data
results = fit_model.transform(test_data)

In [0]:
# Import BinaryClassificationEvaluator to get the ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
# Create evaluator object
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'Survived')

In [0]:
# Preview the transformed test data
results.select('Survived', 'prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [0]:
# Get the ROC/AUC to evaluate the model
AUC = my_eval.evaluate(results)

In [0]:
# Preview the ROC/AUC
AUC

Out[34]: 0.7985875706214689

A perfect model would have a 1.0 ROC/AUC. This model is not terrible but could be better!